In [8]:
import random
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
# Conectar a MongoDB
client = MongoClient("mongodb://localhost:27017/")
database = client["ETIB_TFM"]
collection = database["data_model"]

In [3]:
# Rango de fechas
start_date = datetime(2024, 11, 11)
end_date = datetime(2024, 11, 18)

# Obtener los datos de MongoDB
query = {"Fecha_Transaccion": {"$gte": start_date, "$lt": end_date}}
cursor = collection.find(query)

#Convertir los datos a un DataFrame de Pandas
df_pandas = pd.DataFrame(list(cursor))

# Mostrar las primeras filas del DataFrame
print(df_pandas.head())

                        _id Dispositivo                          Emisor  \
0  67471e2ff909cbe620f2c92c   220005975  (3101000) Bogota Card(Citizen)   
1  67471e2ff909cbe620f2c92d   220005975  (3101000) Bogota Card(Citizen)   
2  67471e21f909cbe620ece0b3   220002978  (3101000) Bogota Card(Citizen)   
3  67471e21f909cbe620ece0b5   220002978  (3101000) Bogota Card(Citizen)   
4  67471e21f909cbe620ece0b6   220002978  (3101000) Bogota Card(Citizen)   

                                  Estacion_Parada    Fase Fecha_Clearing  \
0              (56193) 089A07_TM|089A07_Estaci�AD  Fase 3     2024-11-10   
1              (56193) 089A07_TM|089A07_Estaci�AD  Fase 3     2024-11-10   
2  (57622) 376A00_TM|376A00_Br. Chapinero Central  Fase 3     2024-11-10   
3  (57622) 376A00_TM|376A00_Br. Chapinero Central  Fase 3     2024-11-10   
4  (57622) 376A00_TM|376A00_Br. Chapinero Central  Fase 3     2024-11-10   

    Fecha_Transaccion Hora_Pico_SN ID_Vehiculo       Linea  ...   Valor  \
0 2024-11-11 00:0

In [6]:
# Columnas categóricas a codificar
categorical_columns = ['Ruta', 'Nombre_Perfil', 'Tipo_Tarjeta', 'Estacion_Parada']

# Crear codificadores para cada columna categórica
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df_pandas[col] = le.fit_transform(df_pandas[col])  # Reemplazar la columna con los valores codificados
    label_encoders[col] = le  # Guardar el codificador para cada columna

# Mostrar las primeras filas del DataFrame
df_pandas.head()

,_id,Dispositivo,Emisor,Estacion_Parada,Fase,Fecha_Clearing,Fecha_Transaccion,Hora_Pico_SN,ID_Vehiculo,Linea,...,Valor,archivo,cenefa,Hora,Año,Mes,Día,Fecha,longitud_paradero,latitud_paradero
0,67471e2ff909cbe620f2c92c,220005975,(3101000) Bogota Card(Citizen),890,Fase 3,2024-11-10,2024-11-11 00:01:03,Peak Time,804309,(1138) T11,...,2500.0,Validacion_008_SUMA_20241110.csv,089A07,0,2024,11,11,2024-11-11,-74.083632,4.624574
1,67471e2ff909cbe620f2c92d,220005975,(3101000) Bogota Card(Citizen),890,Fase 3,2024-11-10,2024-11-11 00:01:21,Peak Time,804309,(1138) T11,...,2950.0,Validacion_008_SUMA_20241110.csv,089A07,0,2024,11,11,2024-11-11,-74.083632,4.624574
2,67471e21f909cbe620ece0b3,220002978,(3101000) Bogota Card(Citizen),1133,Fase 3,2024-11-10,2024-11-11 00:03:49,Peak Time,157119,(1064) T13,...,2950.0,Validacion_011_CONSORCIO EXPRESS SAN CRISTOBAL...,376A00,0,2024,11,11,2024-11-11,-74.062612,4.641022
3,67471e21f909cbe620ece0b5,220002978,(3101000) Bogota Card(Citizen),1133,Fase 3,2024-11-10,2024-11-11 00:04:02,Peak Time,157119,(1064) T13,...,2950.0,Validacion_011_CONSORCIO EXPRESS SAN CRISTOBAL...,376A00,0,2024,11,11,2024-11-11,-74.062612,4.641022
4,67471e21f909cbe620ece0b6,220002978,(3101000) Bogota Card(Citizen),1133,Fase 3,2024-11-10,2024-11-11 00:04:09,Peak Time,157119,(1064) T13,...,2950.0,Validacion_011_CONSORCIO EXPRESS SAN CRISTOBAL...,376A00,0,2024,11,11,2024-11-11,-74.062612,4.641022


In [10]:
df_pandas['Ruta'].unique()

array([4, 5, 1, 9, 8, 2, 3, 0, 6, 7])

In [11]:
from scipy.sparse import lil_matrix

# Función para calcular la distancia euclidiana entre dos puntos geográficos (latitud, longitud)
def calcular_distancia(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

# Filtrar los datos para cada ruta
rutas_unicas = df_pandas['Ruta'].unique()  # Obtener las rutas únicas

# Crear un diccionario para almacenar las distancias por ruta
distancias_por_ruta = {}

# Calcular distancias para cada ruta
for ruta in rutas_unicas:
    # Filtrar los paraderos de la ruta
    paraderos_ruta = df_pandas[df_pandas['Ruta'] == ruta][['latitud_paradero', 'longitud_paradero']].to_numpy()
    
    # Crear una matriz dispersa (sólo para distancias no nulas)
    num_paraderos = paraderos_ruta.shape[0]
    distancias = lil_matrix((num_paraderos, num_paraderos))  # Usamos formato disperso
    
    # Calcular las distancias entre los paraderos de esta ruta
    for i in range(num_paraderos):
        for j in range(i+1, num_paraderos):  # Solo calcular la mitad de la matriz
            distancia = calcular_distancia(paraderos_ruta[i, 0], paraderos_ruta[i, 1], 
                                           paraderos_ruta[j, 0], paraderos_ruta[j, 1])
            distancias[i, j] = distancia
            distancias[j, i] = distancia  # La matriz es simétrica
    
    # Guardar las distancias de la ruta
    distancias_por_ruta[ruta] = distancias

# Ejemplo de acceso a las distancias de una ruta
ruta_ejemplo = '1'  # Cambiar por una ruta de tu dataset
distancia_ruta = distancias_por_ruta[ruta_ejemplo].toarray()  # Convertir a matriz densa si es necesario

# Mostrar las primeras filas de la matriz de distancias de la ruta ejemplo
print(distancia_ruta[:10, :10])

KeyboardInterrupt: 